# Statlog German Credit Data

## Add a table of contents
###Introduce the dataset and the purpose of the analysis

In [1]:
import pandas as pd

# 1. Data Ingestion and Exploration

## 1.1 Data Ingestion

#### We will keep it simple for now, and load the data as a csv. We will build an ingestion pipeline for live data later on.

In [ ]:
data = pd.read_csv('')